In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
import tqdm as nootbook_tqdm
from sklearn.ensemble import AdaBoostRegressor


c:\Users\sunrise\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv("cloudy_cluster.csv")
df.head()

,Clearsky DHI,GHI,cluster,Temperature,Clearsky DNI,Relative Humidity,Wind Speed,DNI,Cloud Type,Clearsky GHI,DHI,Solar Zenith Angle
0,0,0,0,5.0,0,63.52,1.4,0,0,0,0,174.62
1,0,0,0,4.7,0,65.31,1.4,0,0,0,0,161.76
2,0,0,0,6.2,0,75.81,1.3,0,0,0,0,148.34
3,0,0,0,5.6,0,79.06,1.3,0,0,0,0,134.93
4,0,0,0,5.0,0,82.37,1.4,0,0,0,0,121.67


In [3]:
df.shape

(5888, 12)

In [4]:
df.isnull().sum()

Clearsky DHI          0
GHI                   0
cluster               0
Temperature           0
Clearsky DNI          0
Relative Humidity     0
Wind Speed            0
DNI                   0
Cloud Type            0
Clearsky GHI          0
DHI                   0
Solar Zenith Angle    0
dtype: int64

In [5]:
df.drop(['cluster'], axis = 1, inplace = True) 

In [6]:
df.describe()

,Clearsky DHI,GHI,Temperature,Clearsky DNI,Relative Humidity,Wind Speed,DNI,Cloud Type,Clearsky GHI,DHI,Solar Zenith Angle
count,5888.000000,5888.000000,5888.000000,5888.000000,5888.000000,5888.000000,5888.000000,5888.000000,5888.000000,5888.000000,5888.000000
mean,22.804008,42.034137,18.194124,82.918648,40.696788,2.417833,71.039402,0.744565,48.393682,21.630605,113.522045
std,49.770999,92.543872,8.220768,180.425605,19.206628,1.116259,164.585380,2.040437,109.771611,46.143014,29.116220
min,0.000000,0.000000,-0.800000,0.000000,6.100000,0.200000,0.000000,0.000000,0.000000,0.000000,19.630000
25%,0.000000,0.000000,11.075000,0.000000,26.630000,1.600000,0.000000,0.000000,0.000000,0.000000,88.857500
50%,0.000000,0.000000,18.700000,0.000000,36.660000,2.200000,0.000000,0.000000,0.000000,0.000000,114.565000
75%,1.250000,1.250000,24.800000,0.000000,51.410000,3.100000,0.000000,0.000000,1.250000,1.250000,134.842500
max,480.000000,390.000000,39.400000,912.000000,100.000000,7.800000,802.000000,8.000000,901.000000,313.000000,174.720000


In [7]:
s1= MinMaxScaler(feature_range=(0,1))
X= s1.fit_transform(df)

In [8]:
s2=MinMaxScaler(feature_range=(0,1))
y= s2.fit_transform(df[['GHI']])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42,shuffle=False)

In [10]:
model = AdaBoostRegressor(n_estimators=100,learning_rate=0.1)
model.fit(X_train, y_train)
print(model)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


AdaBoostRegressor(learning_rate=0.1, n_estimators=100)


In [11]:
import time
start= time.time()
prediction = model.predict(X_test)
end= time.time()
continential= end-start

In [12]:
prediction.shape

(1178,)

In [13]:
prediction = prediction.reshape(prediction.shape[0], 1)

In [14]:
prediction.shape

(1178, 1)

In [15]:
Pred= s2.inverse_transform(prediction)

In [16]:
Actual= s2.inverse_transform(y_test)

In [17]:
Calculated = pd.DataFrame(Pred, columns = ['Prediction'])

In [18]:
Calculated.to_csv(r'E:\CatBoost_1\Johannesburg\AdaBoost\cloudy hours\prediction.csv', index = False)

In [19]:
Actual = pd.DataFrame(Actual, columns = ['Actual'])

In [20]:
Actual.to_csv(r'E:\CatBoost_1\Johannesburg\AdaBoost\cloudy hours\Actual.csv', index = False)

In [21]:
x=Pred
z=Actual

In [22]:
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE
from sklearn.metrics import mean_absolute_percentage_error
MAPE=mean_absolute_percentage_error(z,x)
MAPE

print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)
print("Mean Absolute Percentage Error:",MAPE)

Root Mean Square Error: 3.4487863293471244
Mean Square Error: 11.89412714549161
Mean Absolute Error: 2.1707838538737163
Mean Absolute Percentage Error: 4395340274463845.0


In [23]:
max= x.max()
max

363.72300469483605

In [24]:
min=x.min()
min

1.2689655172413763

In [25]:
NRMSE= (RMSE/(max-min))*100
print("Normalized Root Mean Square Error:",NRMSE)

Normalized Root Mean Square Error: 0.9515099727326514
